In [3]:
import pandas as pd
df = pd.read_csv("Customers.csv", encoding='ISO-8859-1') #file to read using encoding
df.dtypes

CustomerKey     int64
Gender         object
Name           object
City           object
State Code     object
State          object
Zip Code       object
Country        object
Continent      object
Birthday       object
dtype: object

In [4]:
date_formats = ['%d-%m-%Y', '%m/%d/%Y','%d/%m%Y','%m-%d-%Y']

def parse_dates(date_str):
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt)
        except ValueError:
            continue
    return pd.NaT

# Apply the parsing function to the 'date' column
df['Birthday'] = df['Birthday'].apply(parse_dates)

# Convert to the desired format
df['Birthday'] = df['Birthday'].dt.strftime('%d-%m-%Y')

In [85]:
df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,03-07-1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,27-09-1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,26-05-1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,17-09-1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,19-11-1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,25-03-1936
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,16-02-1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,09-11-1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,12-10-1937


In [5]:
df.isnull().sum()

CustomerKey     0
Gender          0
Name            0
City            0
State Code     10
State           0
Zip Code        0
Country         0
Continent       0
Birthday        0
dtype: int64

In [6]:
# Replace NA values with 'NAP' in 'state code' column
df['State Code'] = df['State Code'].fillna('NAP')

df

,CustomerKey,Gender,Name,City,State Code,State,Zip Code,Country,Continent,Birthday
0,301,Female,Lilly Harding,WANDEARAH EAST,SA,South Australia,5523,Australia,Australia,03-07-1939
1,325,Female,Madison Hull,MOUNT BUDD,WA,Western Australia,6522,Australia,Australia,27-09-1979
2,554,Female,Claire Ferres,WINJALLOK,VIC,Victoria,3380,Australia,Australia,26-05-1947
3,786,Male,Jai Poltpalingada,MIDDLE RIVER,SA,South Australia,5223,Australia,Australia,17-09-1957
4,1042,Male,Aidan Pankhurst,TAWONGA SOUTH,VIC,Victoria,3698,Australia,Australia,19-11-1965
...,...,...,...,...,...,...,...,...,...,...
15261,2099600,Female,Denisa Duková,Houston,TX,Texas,77017,United States,North America,25-03-1936
15262,2099618,Male,Justin Solórzano,Mclean,VA,Virginia,22101,United States,North America,16-02-1992
15263,2099758,Male,Svend Petrussen,Wilmington,NC,North Carolina,28405,United States,North America,09-11-1937
15264,2099862,Female,Lorenza Rush,Riverside,CA,California,92501,United States,North America,12-10-1937


In [7]:
df.isnull().sum()

CustomerKey    0
Gender         0
Name           0
City           0
State Code     0
State          0
Zip Code       0
Country        0
Continent      0
Birthday       0
dtype: int64

In [8]:
# Save the cleaned dataset to a new CSV file
df.to_csv('Cleaned_Customers.csv', index=False)

In [9]:
df.dtypes

CustomerKey     int64
Gender         object
Name           object
City           object
State Code     object
State          object
Zip Code       object
Country        object
Continent      object
Birthday       object
dtype: object

In [ ]:
import pandas as pd
import pymysql
# Connect to MySQL
myconnection = pymysql.connect(host='127.0.0.1', user='root', passwd='password', database="project")

# Load CSV data
df = pd.read_csv("Cleaned_Customers.csv")
df = df.dropna()  # Drop null values

# Replace data types from pandas to MySQL
a = ",".join(f"`{i}` {j}" for i, j in zip(df.columns, df.dtypes)).replace("object", "TEXT").replace("int64", "INT")

# Use a valid table name
table_name = "Customers"

# Create the table
create_table_query = f"CREATE TABLE `{table_name}` ({a})"
with myconnection.cursor() as cursor:
    cursor.execute(create_table_query)

# Insert data into the table
for i in range(len(df)):
    insert_query = f"INSERT INTO `{table_name}` VALUES {tuple(df.iloc[i])}"
    with myconnection.cursor() as cursor:
        cursor.execute(insert_query)
    myconnection.commit()

myconnection.close()